# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [1]:
%pip install tavily-python


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [1]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient
import pprint

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# Search query
query = "Get me flights from barcelona to madrid on the January 15th 2026"
tavily.search(query, search_depth="basic")

{'query': 'Get me flights from barcelona to madrid on the January 15th 2026',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.google.com/travel/flights/flights-from-barcelona-to-madrid.html',
   'title': 'Find Cheap Flights from Barcelona to Madrid (BCN - MAD)',
   'content': '* Remove child aged 2 to 11. Add child aged 2 to 11. | Cheapest round-trip flights | $77 | IberiaNonstop2 hrThu, Jan 22 — Mon, Jan 26 | The cheapest round-trip flight from Barcelona to Madrid is currently $77 |  |. | Cheapest business class flights | $350 | IberiaNonstop2 hrThu, Jan 8 — Sun, Jan 11 | The cheapest business class round-trip flight from Barcelona to Madrid is currently $350 |  |. ## Popular airports near Barcelona. 1 hr 20 min is the shortest flight time from Barcelona to Madrid. When should you fly to Madrid, Spain? In terms of flights, Air Europa is the most popular. * Air France offers 58 connecting flights per week. * Tap Air Portugal offers 53 con

In [5]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            },
        },
    },
]

In [6]:
def search_web(query):
    # Initialize Tavily
    tavily = TavilyClient(api_key=TAVILY_API_KEY)
    return "\n".join(result["content"] for result in tavily.search(query, search_depth="basic")["results"])

In [9]:
pprint.pprint(search_web("What is the current weather in Tokyo"))

("{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': "
 "35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': "
 "1765960900, 'localtime': '2025-12-17 17:41'}, 'current': "
 "{'last_updated_epoch': 1765960200, 'last_updated': '2025-12-17 17:30', "
 "'temp_c': 12.4, 'temp_f': 54.3, 'is_day': 0, 'condition': {'text': 'Clear', "
 "'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, "
 "'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 177, 'wind_dir': 'S', "
 "'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': "
 "0.0, 'humidity': 54, 'cloud': 25, 'feelslike_c': 11.2, 'feelslike_f': 52.2, "
 "'windchill_c': 10.3, 'windchill_f': 50.5, 'heatindex_c': 11.6, "
 "'heatindex_f': 52.9, 'dewpoint_c': 2.6, 'dewpoint_f': 36.8, 'vis_km': 10.0, "
 "'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 10.2, 'gust_kph': 16.3}}\n"
 'The temperatures in Tokyo in December are quite cold with temperatures '
 'between 41°F and 53°F


## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [23]:
from datetime import datetime
messages = [
    {"role": "system", "content": f"You are a helpful trip advisor assistant that can search the web to create a plan for a trip. A basic trip requires: 1. Transportation 2. Accommodation 3. Food 4. Activities. Make as many tool calls as needed to create a comprehensive plan. Current date: {datetime.now().strftime('%Y-%m-%d')}"},
    {"role": "user", "content": "Plan a trip to Barcelona for 3 days"}
]

In [24]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [25]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message
pprint.pprint(response.tool_calls)
messages.append(response)

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "web_search":
            print("Searching for", tool_call_arguments)
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": f"{tool_call_arguments["query"]}: {search_results}"})
    print(invoke_model(messages))

else:
    # If there are no tool calls, return the response content
    print(response.content)

[ChatCompletionMessageToolCall(id='call_Zn8kx6hy1xhhSX0QZp2VeFaF', function=Function(arguments='{"query": "Barcelona 3 days transportation options"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_6Dd7T2FumtfTPbVyqePvo5Zr', function=Function(arguments='{"query": "Barcelona 3 days accommodation options"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_NoBWLIL4skvONyDmOwK9voIG', function=Function(arguments='{"query": "Barcelona 3 days food recommendations"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_4u6KWqMLX5utiDrhmvluMzV8', function=Function(arguments='{"query": "Barcelona activities to do in 3 days"}', name='web_search'), type='function')]
Searching for {'query': 'Barcelona 3 days transportation options'}
Searching for {'query': 'Barcelona 3 days accommodation options'}
Searching for {'query': 'Barcelona 3 days food recommendations'}
Searching for {'query': 'Barcelona activities to do 

In [26]:
messages

[{'role': 'system',
  'content': 'You are a helpful trip advisor assistant that can search the web to create a plan for a trip. A basic trip requires: 1. Transportation 2. Accommodation 3. Food 4. Activities. Make as many tool calls as needed to create a comprehensive plan. Current date: 2025-12-17'},
 {'role': 'user', 'content': 'Plan a trip to Barcelona for 3 days'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Zn8kx6hy1xhhSX0QZp2VeFaF', function=Function(arguments='{"query": "Barcelona 3 days transportation options"}', name='web_search'), type='function'), ChatCompletionMessageToolCall(id='call_6Dd7T2FumtfTPbVyqePvo5Zr', function=Function(arguments='{"query": "Barcelona 3 days accommodation options"}', name='web_search'), type='function'), ChatCompletionMessageToolCall(id='call_NoBWLIL4skvONyDmOwK9voIG', function=Function(arguments='{"query": "Barcelona 3 days food recommendati